# Circuit zolder notebook


### TODO:
1. Collect additional accurate coordinates and elevation of the route (3000 data points)
2. Try doing interpolation on the data points to more accurate image
3. Calculate steering radiuses

### Goals:
1. Calculate optimal speed profile for the solar car on the circuit zolder route
2. Additional Parameter to calculate estimated lap energy

In [1]:
import gpxpy
import plotly.express as px
import pandas as pd

In [2]:
# Load GPX file
with open('zolder_filtered.gpx', 'r') as gpx_file:
    gpx = gpxpy.parse(gpx_file)

In [3]:
# Extract data
route_data = []
for track in gpx.tracks:
    for segment in track.segments:
        for point in segment.points:
            route_data.append((point.latitude, point.longitude, point.elevation))

print('Number of points:', len(route_data))

Number of points: 2971


In [9]:
fig = px.line_mapbox(
    lat=[lat for lat, lon, ele in route_data],
    lon=[lon for lat, lon, ele in route_data],
    zoom=14.3
)

fig.update_traces(
    line=dict(width=6, color='blue'),  
)
fig.update_layout(
    mapbox_style="open-street-map",
    width=1200,  
    height=800,
    margin={"r":0,"t":0,"l":0,"b":0}
)
fig.show()


In [24]:
# Show elevation on plotly
elevation = [ele for lat, lon, ele in route_data]
fig = px.line(x=range(len(elevation)), y=elevation, title='Elevation')
fig.show()

In [34]:
from math import radians, sin, cos, sqrt, atan2

def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Calculate the distance between two points on the Earth's surface using the Haversine formula.
    Input: lat1, lon1, lat2, lon2 in decimal degrees
    Output: distance in kilometers
    """
    # Radius of the Earth in kilometers
    R = 6371.0

    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    # Differences between latitudes and longitudes
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Haversine formula
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    # Distance in kilometers
    return R * c

def calculate_total_distance(route):
    """
    Calculate the total distance of a route given a list of (latitude, longitude) pairs.
    Input: route - a list of tuples [(lat1, lon1), (lat2, lon2), ...]
    Output: total distance in kilometers
    """
    total_distance = 0.0

    # Loop through the route and calculate distances between consecutive points
    for i in range(len(route) - 1):
        lat1, lon1 = route[i]
        lat2, lon2 = route[i + 1]
        total_distance += haversine_distance(lat1, lon1, lat2, lon2)

    return total_distance

# usage with filtered_route_data
route = [(lat, lon) for lat, lon, ele in route_data]
total_distance = calculate_total_distance(route)
print(f"Total distance: {total_distance:.2f} km")

Total distance: 4.08 km


In [7]:
# # perform interpolation to get more accurate elevation
# from scipy.interpolate import interp1d
# import numpy as np

# # Extract latitude, longitude, and elevation from the filtered data
# latitude = [lat for lat, lon, ele in route_data]
# longitude = [lon for lat, lon, ele in route_data]
# elevation = [ele for lat, lon, ele in route_data]

# # Create an interpolation function for elevation
# interp_func = interp1d(range(len(elevation)), elevation, kind='linear')

# # Generate a new set of points with interpolated elevation
# new_elevation = interp_func(np.linspace(0, len(elevation) - 1, num=10 * len(elevation)))

In [ ]:
# Convert filtered_route_data to a DataFrame
df = pd.DataFrame(route_data, columns=['latitude', 'longitude', 'elevation'])

# Visualize route in 3D
fig = px.line_3d(df, x='longitude', y='latitude', z='elevation')
fig.update_traces(line=dict(width=20))
fig.update_layout(
    width=1500,  
    height=1000,  
    scene=dict(
        xaxis_title='Latitude',
        yaxis_title='Longitude',
        zaxis_title='Elevation'
    )
)

fig.show()

In [6]:
# # Extract elevations from the original data
# elevation = [ele for lat, lon, ele in route_data]
# filtered_route_data = []
# print("Elevations:", elevation)

# # Filter anomalies where there is a sequence of +0.5 followed by -0.5
# for i in range(1, len(elevation) - 1):  # Loop from the second to the second-to-last point
#     diff1 = elevation[i] - elevation[i - 1]
#     diff2 = elevation[i + 1] - elevation[i]
    
#     # Check if the anomaly condition is met
#     if (diff1 >= 0.2 and diff2 <= -0.2) or (diff1 <= -0.2 and diff2 >= 0.2) or abs(diff1) >= 0.5 or abs(diff2) >= 0.5:
#         print(f"Anomaly detected at index {i} with elevations: {elevation[i - 1]}, {elevation[i]}, {elevation[i + 1]}")
#     else:
#         filtered_route_data.append(route_data[i])
# # Print filtered data
# for lat, lon, ele in filtered_route_data:
#     print(f"Latitude: {lat}, Longitude: {lon}, Altitude: {ele} m")
# print("Number of points: ", len(filtered_route_data))